**Name:** YANG Yongze

**EID:** 58162280

**Kaggle Team Name:** YANG Yongze

# CS5489 - Assignment 1 - SMS Classification

## Final submission
In this file, put the code that generates your final Kaggle submission. It will be used to verify that your Kaggle submission is reproducible.

In [1]:
import numpy as np
%matplotlib inline
import matplotlib_inline   # setup output image format
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
random.seed(100)
from nltk.corpus import stopwords
from nltk import word_tokenize, PorterStemmer, SnowballStemmer, WordNetLemmatizer
import re
import nltk

# Init stop words and stemmer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Define some helper functions
def read_text_data(fname):
    txtdata = []
    classes = []
    with open(fname, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            # get the text
            txtdata.append(row[0])
            # get the class (convert to integer)
            if len(row)>1:
                classes.append(int(row[1]))

    return (txtdata, classes)

def write_csv_kaggle_sub(fname, Y):
    # fname = file name
    # Y is a list/array with class entries

    # header
    tmp = [['Id', 'Prediction']]

    # add ID numbers for each Y
    for (i,y) in enumerate(Y):
        tmp2 = [(i+1), y]
        tmp.append(tmp2)

    # write CSV file
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(tmp)

def preprocess(str):
    """
    Preprocess text data by removing non-alphanumeric characters & stop word and lemmatizing

    :param str: input string
    :return: preprocessed string
    """

    # Remove non-alphanumeric characters
    string_only_alphanumeric = re.sub(r'[^a-zA-Z0-9]', ' ', str)
    # Tokenize the string into individual words
    words = word_tokenize(string_only_alphanumeric)
    new_str = ''

    for word in words:
        if not word.isdigit():
            # Remove stop word
            if word not in stop_words:
                # lemmatization and convert to lower case
                stemmed_word = lemmatizer.lemmatize(word)
                lower_word = stemmed_word.lower()
                if new_str == '':
                    new_str += lower_word
                else:
                    new_str = new_str + ' ' + lower_word
    return new_str

# load the data
(traintxt, trainY) = read_text_data("smishing_train.txt")
(testtxt, _)       = read_text_data("smishing_test.txt")

print(len(traintxt))
print(len(testtxt))

# Preprocess the data
train_txt_preprocessed = [preprocess(s) for s in traintxt]
test_txt_preprocessed = [preprocess(s) for s in testtxt]
print('Preprocessing finished.')

# Count Vectorizer
vectorizer = CountVectorizer(stop_words='english', max_features=3150)
train_features = vectorizer.fit_transform(train_txt_preprocessed)
test_features = vectorizer.transform(test_txt_preprocessed)
print('CountVectorizer feature representing finished.')

# Calculate TF-IDF value
tf_trans = feature_extraction.text.TfidfTransformer(use_idf=True, norm='l1')
train_X = tf_trans.fit_transform(train_features)
test_X = tf_trans.transform(test_features)
print('TF-IDF feature representing calculated.')

# Train BernoulliNB
model_bnb = naive_bayes.BernoulliNB(alpha=0.1)
model_bnb.fit(train_X,trainY)
print('BernoulliNB Trained.')

y_pred_bnb = model_bnb.predict(test_X)
print('Predicting finished')

# Make the Submission CSV file
write_csv_kaggle_sub("final_submission.csv", y_pred_bnb)
# See some predicted data
classnames = unique(trainY)
classlabels = ['normal', 'spam', 'smishing']
for c in classnames:
    tmp = where(trainY==c)
    for a in tmp[0][0:10]:
        print('[{}]: {}'.format(classlabels[trainY[a]], traintxt[a]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yyz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yyz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yyz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\yyz\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


2985
2986
Preprocessing finished.
CountVectorizer feature representing finished.
TF-IDF feature representing calculated.
BernoulliNB Trained.
Predicting finished
[normal]: Dunno da next show aft 6 is 850. Toa payoh got 650.
[normal]: I.ll hand her my phone to chat wit u
[normal]: I dont have i shall buy one dear
[normal]: Nite...
[normal]: Ok�congrats�
[normal]: Oops I did have it,  &lt;#&gt; ?
[normal]: Yeah. I got a list with only u and Joanna if I'm feeling really anti social
[normal]: Message from . I am at Truro Hospital on ext. You can phone me here. as I have a phone by my side
[normal]: Y ü wan to go there? C doctor?
[normal]: Not from this campus. Are you in the library?
[spam]: I'd like to tell you my deepest darkest fantasies. Call me 09094646631 just 60p/min. To stop texts call 08712460324 (nat rate)
[spam]: Santa Calling! Would your little ones like a call from Santa Xmas eve? Call 09058094583 to book your time.
[spam]: Meet Top 35 US universities in Delhi at India Habitat